origin: http://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html  
translator: Hongpu Liu

In [1]:
%matplotlib inline

# PyTorch简介
## 1. Torch张量库的介绍
所有的深度学习都在对张量进行计算，张量可以看成是矩阵的推广，通常张量的维度大于2维。后面我们将深入的了解到其含义，首先看一下可以对张量做什么。

In [2]:
from __future__ import print_function

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### 1.1 创建张量
张量可以通过调用**torch.Tensor()**从Python列表来创建。

In [3]:
# 根据给定的数据创建一个torch.Tensor对象，它是一个1D向量
V_data = [1., 2., 3.]
V = torch.Tensor(V_data)
print(V)

# 创建一个矩阵
M_data = [[1., 2., 3.], [4., 5., 6.]]
M = torch.Tensor(M_data)
print(M)

# 创建一个2x2x2的3D张量
T_data = [[[1., 2.], [3., 4.]],
          [[5., 6.], [7., 8.]]]
T = torch.Tensor(T_data)
print(T)


 1
 2
 3
[torch.FloatTensor of size 3]


 1  2  3
 4  5  6
[torch.FloatTensor of size 2x3]


(0 ,.,.) = 
  1  2
  3  4

(1 ,.,.) = 
  5  6
  7  8
[torch.FloatTensor of size 2x2x2]



什么是3D张量？如果你有一个向量，向量的元素是一个标量。如果你有一个矩阵，矩阵的元素是一个向量。如果你有一个3D张量，它的每一个元素都是一个矩阵！

**术语**：当我们说“张量”时，它指的是任何的**torch.Tensor**对象。矩阵和向量是**torch.Tensor**的特殊情况，它们的维度分别是2和1.当我们讨论3D张量时，我们会显式的使用术语“3D张量”。

In [4]:
# V的元素是一个标量
print(V[0])

# M的元素是一个向量
print(M[0])

# T的元素是一个矩阵
print(T[0])

1.0

 1
 2
 3
[torch.FloatTensor of size 3]


 1  2
 3  4
[torch.FloatTensor of size 2x2]



你可以创建其他数据类型的张量。默认情况下是**Float**类型。要创建一个整数类型的张量，可以使用**torch.LongTensor()**。查看文档可以看到更多的数据类型，但是**Float**和**Long**是最常用的。

你可以用**torch.randn()**来创建包含随机数据的张量，函数的参数是张量的维度。

In [5]:
x = torch.randn((3, 4, 5))
print(x)


(0 ,.,.) = 
  0.6614  0.2669  0.0617  0.6213 -0.4519
 -0.1661 -1.5228  0.3817 -1.0276 -0.5631
 -0.8923 -0.0583 -0.1955 -0.9656  0.4224
  0.2673 -0.4212 -0.5107 -1.5727 -0.1232

(1 ,.,.) = 
  3.5870 -1.8313  1.5987 -1.2770  0.3255
 -0.4791  1.3790  2.5286  0.4107 -0.9880
 -0.9081  0.5423  0.1103 -2.2590  0.6067
 -0.1383  0.8310 -0.2477 -0.8029  0.2366

(2 ,.,.) = 
  0.2857  0.6898 -0.6331  0.8795 -0.6842
  0.4533  0.2912 -0.8317 -0.5525  0.6355
 -0.3968 -0.6571 -1.6428  0.9803 -0.0421
 -0.8206  0.3133 -1.1352  0.3773 -0.2824
[torch.FloatTensor of size 3x4x5]



### 1.2 张量的运算
你可以按照你期望的方式来操作张量。

In [6]:
x = torch.Tensor([1., 2., 3.])
y = torch.Tensor([4., 5., 6.])
z = x + y
print(z)


 5
 7
 9
[torch.FloatTensor of size 3]



张量操作的完整列表可以查阅[文档](http://pytorch.org/docs/torch.html)，你会看到大量的张量运算。张量的运算不仅仅支持数学运算。

后面我们会常用的一个操作是数据的串联。

In [7]:
# 默认情况下，串联数据发生在第一个轴（按行串联）
x_1 = torch.randn(2, 5)
y_1 = torch.randn(3, 5)
z_1 = torch.cat([x_1, y_1])
print(z_1)

# 按列串联
x_2 = torch.randn(2, 3)
y_2 = torch.randn(2, 5)
# 第二个参数指定了进行拼接的轴
z_2 = torch.cat([x_2, y_2], 1)
print(z_2)

# 如果拼接的张量维度不匹配，将会出错。把下面语句的注释取消可以看到错误
# torch.cat([x_1, x_2])


-2.5667 -1.4303  0.5009  0.5438 -0.4057
 1.1341 -1.1115  0.3501 -0.7703 -0.1473
 0.6272  1.0935  0.0939  1.2381 -1.3459
 0.5119 -0.6933 -0.1668 -0.9999 -1.6476
 0.8098  0.0554  1.1340 -0.5326  0.6592
[torch.FloatTensor of size 5x5]


-1.5964 -0.3769 -3.1020 -0.0020 -1.0952  0.6016  0.6984 -0.8005
-0.0995 -0.7213  1.2708  1.5381  1.4673  1.5951 -1.5279  1.0156
[torch.FloatTensor of size 2x8]



### 1.3 改变张量的形状
使用**.view()**方法可以改变张量的形状。由于很多神经网络的组件需要特定的输入形状，因此这个方法会被频繁的使用。经常在传递数据给组件之前需要改变数据的形状。

In [8]:
x = torch.randn(2, 3, 4)
print(x)
print(x.view(2, 12)) # 改变为2行、12列的张量

# 下面与上面的代码结果一样，如果其中一个维度为-1，该维度的值会自动进行计算
print(x.view(2, -1))


(0 ,.,.) = 
 -0.2020 -1.2865  0.8231 -0.6101
 -1.2960 -0.9434  0.6684  1.1628
 -0.3229  1.8782 -0.5666  0.4016

(1 ,.,.) = 
 -0.1153  0.3170  0.5629  0.8662
 -0.3528  0.3482  1.1371 -0.3339
 -1.4724  0.7296 -0.1312 -0.6368
[torch.FloatTensor of size 2x3x4]



Columns 0 to 9 
-0.2020 -1.2865  0.8231 -0.6101 -1.2960 -0.9434  0.6684  1.1628 -0.3229  1.8782
-0.1153  0.3170  0.5629  0.8662 -0.3528  0.3482  1.1371 -0.3339 -1.4724  0.7296

Columns 10 to 11 
-0.5666  0.4016
-0.1312 -0.6368
[torch.FloatTensor of size 2x12]



Columns 0 to 9 
-0.2020 -1.2865  0.8231 -0.6101 -1.2960 -0.9434  0.6684  1.1628 -0.3229  1.8782
-0.1153  0.3170  0.5629  0.8662 -0.3528  0.3482  1.1371 -0.3339 -1.4724  0.7296

Columns 10 to 11 
-0.5666  0.4016
-0.1312 -0.6368
[torch.FloatTensor of size 2x12]



## 2. 计算图与自动微分
计算图是提高深度学习编程的效率的核心，因为它允许你无需自己编写反向梯度传播。计算图指定了你的数据是如何组合运算得到输出的。由于计算图完整包含了运算所涉及的参数，因此它为计算微分提供了充足的信息。这听上去含糊不清，因此让我们看一下在使用PyTorch的基础类**autograd.Variable**时发生了什么。

首先从程序员的视角来看。我们之前创建的**torch.Tensor**到底存储了什么？显然包括数据和形状，以及其他一些东西。但是当我们把两个张量进行相加时，得到了输出张量。这个输出张量只知道它的数据和形状。它并不知道自己是两个其他张量的和（因为可能是从文件中读取到的，也可能是其他运算你的结果）。

**Variable**可以对它是如何创建的保持跟踪。

In [9]:
# 变量包含了张量对象
x = autograd.Variable(torch.Tensor([1., 2., 3.]), requires_grad=True)

# 可以通过 .data 属性来访问变量中的数据
print(x.data)

# 你也可以把那些用于张量的运算用于变量
y = autograd.Variable(torch.Tensor([4., 5., 6.]), requires_grad=True)
z = x + y
print(z.data)

# 但是z保存了更多的信息
print(z.grad_fn)


 1
 2
 3
[torch.FloatTensor of size 3]


 5
 7
 9
[torch.FloatTensor of size 3]



因此变量知道自己是如何被创建的。z知道自己不是从文件中读取的，也不是乘法或指数运算的结果。沿着**z.grad_fn**将找到x和y。

但是它是如何帮助我们计算梯度的呢？

In [10]:
# 让我们把z的所有项进行求和
s = z.sum()
print(s)
print(s.grad_fn)

Variable containing:
 21
[torch.FloatTensor of size 1]



s关于x的第一个元素的微分是什么？用数学表达式可写为：
$$\frac{\partial s}{\partial x_0}$$

现在**s**知道它是**z**所有元素的和。而**z**指导它是**x**和**y**的和。也就是：

$$s = \overbrace{x_0 + y_0}^\text{$z_0$} + \overbrace{x_1 + y_1}^\text{$z_1$} + \overbrace{x_2 + y_2}^\text{$z_2$}$$

**s**中包含了足够的信息来确定我们要求的微分是**1**！

当然这掩盖了实际上计算微分的难度。重要的是知道，变量保存了足够的信息用于计算梯度。实际上，Pytorch的开发者在编写**sum()**和**+**运算时，知道如何计算它们的梯度以及如何运行反向传播算法。更深入的讨论超出了本教程的范围。

接下来让Pytorch来计算梯度（注意，如果运行多次的话，梯度将会递增。这是因为Pytorch会把梯度**累加**到**.grad**属性中，这对许多模型非常方便）。

In [11]:
# 调用任意变量的 .backward() 将会运行反向传播
s.backward()
print(x.grad)

Variable containing:
 1
 1
 1
[torch.FloatTensor of size 3]



理解下面代码发生了什么，对于成为一个成功深度学习开发者非常重要。

In [12]:
x = torch.randn((2, 2))
y = torch.randn((2, 2))
z = x + y # 这些是Tensor，无法进行反向传播

var_x = autograd.Variable(x, requires_grad=True)
var_y = autograd.Variable(y, requires_grad=True)
# var_z包含了足够信息来计算梯度
var_z = var_x + var_y
print(var_z.grad_fn)

var_z_data = var_z.data # 获得var_z中封装的张量对象
# 将张量重新封装到一个变量中
new_var_z = autograd.Variable(var_z_data)

# new_var_z是否拥有反向传播计算x和y梯度的信息呢？
# 没有！
print(new_var_z.grad_fn)
# 因为我们把Tensor对象从var_z中取出，该Tensor并不包含它是如何被计算出来的信息。
# 我们把这个Tensor传递给new_var_z，这是new_var_z获得的唯一信息。由于var_z_data
# 并不知道自己是被如何计算出来的，new_var_z同样不会知道。
# 本质上，我们打断了变量的运算历史。

None


下面的规则，对于用**autograd.Variable**来进行计算非常重要（注意：**autograd.Variable**在其他的主要深度学习工具集中都有等价的对象）：

> 如果要将损失函数的误差传播到网络中的一个组件，你就**绝对不能**打断从该组件到损失变量之间的计算链。如果你这样做了，损失将不知道该组件的存在，组件的权重也就无法更新。

上面的错误会悄悄地影响你的代码的执行结果，而且不会导致代码崩溃或警告，因此必须谨慎对待。